# DataFrames Basics Exercises

## Prerrequisites

Install Spark and Java in VM

In [ ]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.1
!wget -q https://apache.osuosl.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz

In [ ]:
ls -l # check the .tgz is there

total 267684
drwxr-xr-x 1 root root      4096 Dec  6 14:35 sample_data/
-rw-r--r-- 1 root root 274099817 Oct 15 10:53 spark-3.3.1-bin-hadoop2.tgz


In [ ]:
# unzip it
!tar xf spark-3.3.1-bin-hadoop2.tgz

In [ ]:
!pip install -q findspark

Defining the environment

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop2"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

Start Spark Session

---

In [ ]:
import findspark
findspark.init("spark-3.3.1-bin-hadoop2")# SPARK_HOME

from pyspark.sql import SparkSession

# create the session
spark = SparkSession \
        .builder \
        .appName("DataFramesBasics Exercises") \
        .master("local[*]") \
        .getOrCreate()

spark.version

'3.3.1'

In [ ]:
spark

In [ ]:
# For Pandas conversion optimization
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [ ]:
# Import sql functions
from pyspark.sql.functions import *

Download datasets

In [ ]:
!mkdir -p dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2022/master/datasets/movies.json -P /dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2022/master/datasets/cars.json -P /dataset

## DataFrames Basics Exercises

1) Create a manual DF describing smartphones
  - maker
  - model
  - screen dimension
  - camera megapixels
  
2) Read another file from the dataset/ folder, e.g. movies.json
  - print its schema
  - count the number of rows, call count()

In [ ]:
# 1)
schema = "`maker` STRING NOT NULL, `model` STRING NOT NULL, `screen dimension` FLOAT NOT NULL, `camera megapixels` INTEGER NOT NULL"
data = [("abc", "123", 32.3, 48), 
        ("cdv", "194", 25.3, 18)
        ]
df = spark.createDataFrame(data, schema)
df.show()

In [ ]:
# 2)
df2 = spark.read.json("/dataset/movies.json")
df2.printSchema()
print(f'number of rows: {df2.count()}')

## Columns and Expressions Exercises

1. Read the movies DF and select 2 columns of your choice
2. Create another column summing up the total profit of the movies = US_Gross + Worldwide_Gross + DVD sales. Are you pbtaining nulls? How you can solve it?
3. Select all COMEDY movies with IMDB rating above 6

Use as many versions as possible

In [ ]:
# 1)
df_movies = spark.read.json("/dataset/movies.json")
df_movies.select("Director","Title").show()

In [ ]:
# 2)
df_movies \
.na.fill(value=0,subset=["US_Gross", "Worldwide_Gross", "US_DVD_Sales"]) \
.withColumn("Total Profit",col("US_Gross")+col("Worldwide_Gross")+ col("US_DVD_Sales")) \
.select("Title", "Total Profit").show()

In [ ]:
# 3)
df_movies.filter((col("Major_Genre") == 'Comedy') & (col('IMDB_Rating') > 6)).show()

In [ ]:
# 3) alternativa
df_movies.filter("Major_Genre == 'Comedy' and IMDB_Rating > 6").show()